# 初步的构思

1、线上数据集

注意618的影响  
领取优惠券的时间接近周末？    
领取消费券的时间接近？    
领取优惠券的对应商家距离近？  
领取优惠券的力度大？（门槛低，金额高）  
领取优惠券的时间越近消费的概率越高？  
领取消费券的时间，靠近放假时间？  
领取优惠券越多，核销的概率越大？  
不同折扣，不同优惠方式 ，哪种核销概率高？
领取到核销的时间特征  
领券时间和用券消费的时间间隔有什么特征，为什么预测的是“15天”

人本身消费属性？（RFM，结合线上数据来定义消费者属性）  
人和固定商家有消费关系？(喜欢这个商家)  
线下偏好、线上偏好  
被营销偏好  
优惠券id,不同人，有一样的吗？

商家，虽然线上下编号无法对应，但是各自是否有规律  
商品本身费用高？  
商品本身消费的人群就少？  
商家品质不咋地，或者本身不善于这种营销？（过往成绩就不好）  


线上优惠券消费，和线下优惠券核销，可能不是一种消费习惯，需要鉴别  
领优惠券的特征，有哪些与核销相关？  

In [1]:
import sys 
sys.path.append('./app/')
import fileio
import preprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 读取数据
dfoff ,dftest,dfon = fileio.read_all('./data/')

# 特征处理
dfoff = preprocess.get_label(dfoff)
dfoff = preprocess.process_manjian(dfoff)


#暂时用不上的
# dfoff,weekdaycols = preprocess.process_Weekday(dfoff)

# td = pd.to_datetime(dfoff['Date'], format='%Y%m%d') -  pd.to_datetime(dfoff['Date_received'], format='%Y%m%d')
# dfoff.loc[:,'核销时间差'] = [i.days for i in td]


In [3]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,discount_rate,discount_man,discount_jian,discount_type
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,1.000000,0,0,NaN
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,0,0.866667,150,20,1.0
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,0.950000,20,1,1.0
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,0.950000,20,1,1.0
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN,0,0.950000,20,1,1.0


In [4]:
dfoff_lab_n = dfoff[dfoff.label<0]# 未领券
dfoff_lab_01 = dfoff[dfoff.label>=0]#领券
dfoff_lab_1 = dfoff[dfoff.label==1]#核销
dfoff_lab_0 = dfoff[dfoff.label==0]#领券不用

# 核销了的优惠券有哪些特征呢

In [5]:
# dfoff_lab_1.核销时间差.value_counts().sort_index()
print('领券后一天核销数最高，然后指数递减')

领券后一天核销数最高，然后指数递减


# 1 user_id
人的消费属性（领券次数，核销次数，核销率，满减特征，商店特征，领券特征，）

In [6]:
# 有核销的人，他们各核销了多少张券
list_1 = dfoff_lab_1.User_id.value_counts()
list_1.head()

2751537    118
6641735    113
6929894     93
2839484     71
501441      69
Name: User_id, dtype: int64

In [7]:
user_off_list = dfoff.User_id.unique()

## 1.1 001是核销最多的用户
领过券的商户数量只有两家,一家只领取过一次。也许 用户-商家，融合后再做特征会好一些，(某些用户会偏爱某个商家)  
唯一没有核销的，距离为4  
领券日期，比较平均
做了新特征【领券未用 	核销数 	未领直接买 	领券数】【是否有核销行为】

### 1.1.1领券核销数量以及被核销商家
做了新特征【领券未用 	核销数 	未领直接买 	领券数】，可以进一步组合

In [10]:
def user_label_count(userid):
    lcdict = {0:0,1:0,-1:0}
    df001 = dfoff[dfoff.User_id==userid]
    dflabelcount = df001.label.value_counts()
    for i in dflabelcount.index:
        lcdict[i] = dflabelcount[i]
        
    dftest = pd.DataFrame.from_dict(lcdict,orient='index').T
    dftest.columns = (['领券未用','核销数','未领直接买'])
    dftest.loc[0,'领券数'] = dftest.loc[0,['领券未用','核销数']].sum()
    
    return dftest

In [11]:
def user_merchant_count(userid):
    df001 = dfoff[dfoff.User_id==userid]
    dfMerchantcount = df001.Merchant_id.value_counts()
    mermode = dfMerchantcount.max()
    mernum = dfMerchantcount.count()
    return dfMerchantcount,mermode,mernum

In [ ]:
df001 = dfoff[dfoff.User_id==list_1.index[1]]
df002 = dfoff[dfoff.User_id==list_1.index[1]]

In [155]:
n=0
ll = []
for i in list_1.index:
    if n%1000 == 0 :
        print(n)
    a = user_label_count(i)
    _,b,c = user_merchant_count(i)
    a.loc[:,'核销店铺数'] = c
    a.loc[:,'单店铺最大消费数量'] = b
    ll.append(a)
    n=n+1

    #print("="*10,'\n',a)
dftest = pd.concat(ll)

0


KeyboardInterrupt: 

In [ ]:
# 做userid的特征

In [95]:
dftest.核销店铺数.value_counts()

1.0     32
2.0     26
3.0     22
4.0      7
6.0      3
7.0      3
5.0      2
11.0     2
13.0     1
14.0     1
8.0      1
Name: 核销店铺数, dtype: int64

In [100]:
dftest

,领券未用,核销数,未领直接买,领券数,核销店铺数,单店铺最大消费数量
0,1,118,36,119.0,2.0,154.0
0,7,113,12,120.0,3.0,127.0
0,2,93,7,95.0,1.0,102.0
0,12,71,27,83.0,11.0,94.0
0,4,69,40,73.0,1.0,113.0
0,0,56,4,56.0,1.0,60.0
0,3,54,54,57.0,2.0,73.0
0,7,41,0,48.0,3.0,46.0
0,4,40,17,44.0,2.0,60.0
0,3,38,17,41.0,2.0,57.0


In [12]:
user_off_list = dfoff.User_id.unique()
print(len(user_off_list))

539438


In [149]:
n = 0
ll = []
for uid in  user_off_list:
    if n%1000==0:
        print(n)
    ll.append(user_features(uid))
    n = n+1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000


KeyboardInterrupt: 

In [150]:
pd.DataFrame(ll)

,0,1,2,3,4,5
0,1439408,5,0,2,6,2
1,1832624,1,0,0,1,1
2,2029232,3,0,2,3,3
3,2747744,1,0,0,1,1
4,196342,1,0,1,2,1
5,2223968,1,0,0,1,1
6,73611,1,0,0,1,1
7,163606,1,0,0,1,1
8,3273056,1,0,0,1,1
9,94107,1,0,0,1,1


In [18]:
def user_features(uid):
    """返回userid，只领券，核销数，直接买，单商铺最大购买次数，成交商家数"""
    lcdict = {0:0,1:0,-1:0}
    df001 = dfoff[dfoff.User_id==uid]
    
    dflabelcount = df001.label.value_counts()
    dfMerchantcount = df001.Merchant_id.value_counts()
    for i in dflabelcount.index:
        lcdict[i] = dflabelcount[i]
    
    return uid,lcdict[0],lcdict[1],lcdict[-1],dfMerchantcount.max(),dfMerchantcount.count()

In [144]:
【购买（直接买，领券买），不购买（只领券）】

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,discount_rate,discount_man,discount_jian,discount_type,核销时间差
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,1.000000,0,0,NaN,NaN
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,0,0.866667,150,20,1.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,0.950000,20,1,1.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,0.950000,20,1,1.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN,0,0.950000,20,1,1.0,NaN
5,1439408,2632,NaN,NaN,0.0,NaN,20160516.0,-1,1.000000,0,0,NaN,NaN
6,1439408,2632,8591.0,20:1,0.0,20160516.0,20160613.0,0,0.950000,20,1,1.0,28.0
7,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,0.900000,200,20,1.0,NaN
8,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,0.900000,200,20,1.0,NaN
9,2029232,450,1532.0,30:5,0.0,20160530.0,NaN,0,0.833333,30,5,1.0,NaN


In [135]:
pd.DataFrame([[0,1,2,3,],[0,2,3,4]])

,0,1,2,3
0,0,1,2,3
1,0,2,3,4


In [ ]:
def user_merchant_count(userid):
    df001 = dfoff[dfoff.User_id==userid]
    dfMerchantcount = df001.Merchant_id.value_counts()
    mermode = dfMerchantcount.max()
    mernum = dfMerchantcount.count()
    return dfMerchantcount,mermode,mernum

In [ ]:
def user_label_count(userid):
    lcdict = {0:0,1:0,-1:0}
    df001 = dfoff[dfoff.User_id==userid]
    dflabelcount = df001.label.value_counts()
    for i in dflabelcount.index:
        lcdict[i] = dflabelcount[i]
        
    dftest = pd.DataFrame.from_dict(lcdict,orient='index').T
    dftest.columns = (['领券未用','核销数','未领直接买'])
    dftest.loc[0,'领券数'] = dftest.loc[0,['领券未用','核销数']].sum()
    
    return dftest

### 1.1.2 满减特征
001没发现啥规律  
不过，核销和总体的满减分布确实不一样

In [24]:
df001.discount_man.value_counts()

30     109
0       40
20       4
200      1
10       1
Name: discount_man, dtype: int64

In [25]:
dfoff.discount_man.value_counts().sort_index()

0      734874
5        2526
10      43767
20     143232
30     301082
50      87397
100    242411
150     23729
200    146602
300     29264
Name: discount_man, dtype: int64

In [26]:
dfoff_lab_1.discount_man.value_counts().sort_index()

0       3852
5        563
10      6343
20     18804
30     22731
50      4217
100     5755
150      343
200     1504
300      283
Name: discount_man, dtype: int64

### 1.1.3 领券日期以及核销日期
较为平均  

In [27]:
df001.Date_received.value_counts().sort_index()

20160125.0    1
20160229.0    1
20160301.0    1
20160322.0    3
20160325.0    2
20160326.0    1
20160422.0    2
20160424.0    3
20160425.0    3
20160426.0    1
20160427.0    3
20160428.0    4
20160430.0    4
20160501.0    5
20160503.0    2
20160506.0    5
20160507.0    1
20160509.0    4
20160510.0    6
20160512.0    6
20160513.0    4
20160515.0    4
20160516.0    3
20160518.0    5
20160519.0    4
20160521.0    2
20160522.0    2
20160524.0    2
20160525.0    4
20160527.0    5
20160528.0    4
20160530.0    1
20160531.0    3
20160602.0    4
20160603.0    3
20160605.0    1
20160606.0    1
20160614.0    6
20160615.0    3
Name: Date_received, dtype: int64

In [28]:
df001

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,discount_rate,discount_man,discount_jian,discount_type,核销时间差
400243,2751537,7910,NaN,NaN,0.0,NaN,20160403.0,-1,1.000000,0,0,NaN,NaN
400244,2751537,7910,NaN,NaN,0.0,NaN,20160406.0,-1,1.000000,0,0,NaN,NaN
400245,2751537,7910,NaN,NaN,0.0,NaN,20160331.0,-1,1.000000,0,0,NaN,NaN
400246,2751537,7910,11637.0,30:5,0.0,20160424.0,20160424.0,1,0.833333,30,5,1.0,0.0
400247,2751537,7910,9117.0,20:5,0.0,20160325.0,20160328.0,1,0.750000,20,5,1.0,3.0
400248,2751537,7910,9117.0,20:5,0.0,20160325.0,20160328.0,1,0.750000,20,5,1.0,3.0
400249,2751537,7910,11637.0,30:5,0.0,20160424.0,20160424.0,1,0.833333,30,5,1.0,0.0
400250,2751537,7910,11637.0,30:5,0.0,20160422.0,20160424.0,1,0.833333,30,5,1.0,2.0
400251,2751537,7910,11637.0,30:5,0.0,20160427.0,20160427.0,1,0.833333,30,5,1.0,0.0
400252,2751537,7910,11637.0,30:5,0.0,20160427.0,20160427.0,1,0.833333,30,5,1.0,0.0


In [33]:
user_label_count(list_1.index[1])

,领券未用,核销数,未领直接买,领券数
0,7,113,12,120.0
